In [1]:
import random
import re
from collections import defaultdict
from pathlib import Path
import networkx as nx
from manim import *
from selma import BACKGROUND
from selma.graph import MGraph, gvlayout_factory, test_draw
from selma.mklm import MarkovChainTextGenerator, clean_text
from pathlib import Path

from markov import next_weight, mk_rnd_next, generate, generate0, build_markov_chain, next_weight, mk_rnd_next, generate, next_weight0, tokenize


config.background_color = BACKGROUND

In [2]:
corpora = {
  'mc': """
Markov chains are mathematical systems that undergo transitions from one state to another 
on a state space. A sequence of possible events in which the probability of each event 
depends only on the state attained in the previous event is called a Markov chain.
""",
  'ttls': """
Twinkle, twinkle, little star,
How I wonder what you are!
Up above the world so high,
Like a diamond in the sky.
""",
  'gadda': 'Come poco pepe a coppia cuoce e scoppia',  # Gadda
  'giovanni': 'In principio era il Verbo, e il Verbo era presso Dio, e il Verbo era Dio',  # dal Vangelo di Giovanni, 1:1
  'commedia': Path('../../data/commedia.txt').read_text(),
  'promessi': Path('../../data/papini.txt').read_text(),
}

In [3]:
results = {}

In [4]:
%%manim --hide-splash -qm -v WARNING MarkovByChar

class MarkovByChar(Scene):
  def construct(self):
    results['gadda2'] = build_markov_chain(self, tokenize(corpora['gadda'], by_char=True), layout = gvlayout_factory('neato', heightscale=.5), node_scale=.8, order = 2)

In [19]:
%%manim --hide-splash -qm -v WARNING MarkovByWord

class MarkovByWord(Scene):
  def construct(self):
    results['giovanni1'] = build_markov_chain(self, tokenize(corpora['giovanni'], by_char=False), layout = gvlayout_factory('neato', heightscale=.6), node_scale=.6, order = 1)


In [11]:
nw0 = next_weight0(
  tokenize(clean_text(corpora['giovanni']), by_char=False),
)
' '.join(generate0(mk_rnd_next(nw0, 42), 20))

'il verbo dio principio il il in verbo presso verbo principio e verbo principio il e principio e era verbo'

In [22]:
'pino'.split()[0]

'pino'

In [ ]:
def random_walk(scene, weight, start, num, seed = None):
  G = nx.DiGraph(set(weight.keys()))
  G.remove_edges_from(nx.selfloop_edges(G))
  MG = MGraph(G, layout=gvlayout_factory('neato', heightscale=.5), node_scale=.8)
  for s, t in G.edges():
    me = MG.medge(s, t)
    me.z_index = 0
    me.set_stroke(width=weight[(s, t)] * 2)
  scene.add(MG.mgraph)
  rn = mk_rnd_next(next_weight(weight), seed)
  gen = generate(rn, start, num)
  T = Tex(*[ts for t in [g.split()[0] for g in gen] for ts in (t, ' ')], color=BACKGROUND).scale(0.9)
  T.to_edge(UL)
  scene.add(T)
  
  s = gen[0]
  
  dot = Dot(color=PURE_GREEN)
  dot.move_to(MG.mnode(gen[0]))
  scene.add(dot)
  start_pos = len(gen[0].split())
  T[:start_pos + 1].set_color(PURE_GREEN)
  scene.wait(.5)
  for pos, t in enumerate(gen[1:], 1):
    T[:pos*2].set_color(BLACK)
    T[pos*2].set_color(PURE_GREEN)
    if s != t: 
      scene.play(MG.movealong(dot, s, t), run_time=.5)
    s = t


In [27]:
%%manim --hide-splash -qm -v WARNING GenerateG2

class GenerateG2(Scene):
  def construct(self):
    random_walk(self, results['gadda1'], 'o p', 10, seed = 42)


KeyError: 'gadda1'

In [30]:
%%manim --hide-splash -qm -v WARNING GenerateG1

class GenerateG1(Scene):
  def construct(self):
    random_walk(self, results['giovanni1'], 'il', 10, seed = 42)
